In [1]:
!dinotool ../test/data/sintel2.mp4 -o vis_demo --batch-size 4 --save-features 'frame'

Using cache found in /home/mikko/.cache/torch/hub/facebookresearch_dinov2_main
Using model: dinov2_vits14_reg
Input size: (1274, 546)
Feature map size: (91, 39)
Extracting frame-level features. This does not produce a video output.
100%|███████████████████████████████████████████| 30/30 [00:27<00:00,  1.11it/s]


In [2]:
import pandas as pd
df = pd.read_parquet("vis_demo.parquet")
df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_374,feature_375,feature_376,feature_377,feature_378,feature_379,feature_380,feature_381,feature_382,feature_383
0,1.226998,-0.111191,-3.338483,-0.024638,-0.631239,1.383679,-1.508697,1.620837,-0.022249,0.558359,...,-3.054588,0.885869,-0.155574,-0.385375,-0.132694,1.635442,-1.745795,-0.469182,-1.202406,0.108280
1,1.283252,-0.339608,-3.384149,0.239383,-0.707855,1.637160,-1.371985,1.531486,-0.098427,0.698198,...,-2.928041,0.742765,-0.158181,-0.257814,-0.073360,1.408982,-2.088218,-0.769860,-1.146614,0.027956
2,0.922155,-0.005924,-3.463403,0.056782,-0.429475,1.801999,-1.279644,1.207272,-0.137652,0.765128,...,-2.763318,0.882392,-0.182768,-0.472445,0.052268,1.252255,-1.688350,-0.710287,-1.129862,0.233970
3,0.920486,-0.304236,-3.081512,-0.103470,-0.728669,1.494304,-1.111631,1.163849,-0.313035,0.929783,...,-2.879076,0.832597,-0.148794,-0.358394,0.034945,1.771147,-1.576589,-0.740988,-1.171328,0.061616
4,0.962223,-0.407951,-3.115240,-0.477747,-0.610310,1.578843,-1.270674,1.700094,-0.465011,0.535229,...,-2.914624,0.967154,-0.256062,-0.185420,-0.011835,1.705593,-1.899136,-0.620152,-0.967431,-0.370488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2.750598,-1.418642,-1.355923,-0.172433,-0.878254,2.163900,-1.457821,0.747625,-0.776769,1.389399,...,-1.171976,-1.152629,0.499474,0.281162,-0.448137,2.565470,-0.843433,1.367541,-1.585475,-0.438883
116,2.672098,-1.092770,0.036669,-0.110369,-1.046606,1.732031,-1.704710,1.121665,-0.296431,1.657832,...,-0.856266,-1.024648,-0.251471,-0.020957,-0.265338,1.854379,-1.269866,0.901349,-1.213509,-0.651088
117,1.991491,-0.929374,0.220060,0.062209,-0.951488,0.593437,-0.861540,1.845629,-0.214706,1.647444,...,-0.769649,-0.925001,0.560903,0.106492,-0.635497,1.271770,-1.453244,0.428481,-1.096014,-1.509625
118,2.263419,-0.782159,-0.638500,0.487778,-0.952429,0.841630,-1.659946,0.370305,-0.085564,0.939508,...,-1.550555,-0.462710,0.397200,0.502313,-0.093229,1.892890,-1.234376,-0.114848,-1.796174,-2.267105


In [3]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
coords = pca.fit_transform(df)

In [4]:
coords.max(axis=0), coords.min(axis=0)

(array([10.844045,  4.806987], dtype=float32),
 array([-12.617585, -11.515226], dtype=float32))

In [5]:
!mkdir vis
!mkdir vis_original
!mkdir vis_combo

In [6]:
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

for i in tqdm(range(coords.shape[0])):
    plt.figure()
    plt.plot(coords[:i, 0], coords[:i, 1], '-o', color='blue')
    plt.xlim((-13, 15))
    plt.ylim((-13, 15))
    plt.savefig(f"vis/{i:04d}.jpg")
    plt.close()


  0%|          | 0/120 [00:00<?, ?it/s]

In [7]:
!ffmpeg -i ../test/data/sintel2.mp4 -q:v 2 -start_number 0 vis_original/'%05d.jpg'

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [8]:
import numpy as np
from PIL import Image
for i in tqdm(range(coords.shape[0])):
    frame = Image.open(f"vis_original/{i:05d}.jpg")
    plot = Image.open(f"vis/{i:04d}.jpg")
    plot = plot.resize(frame.size)

    combined = np.vstack((np.array(frame), np.array(plot)))
    combined = Image.fromarray(combined)
    combined.save(f"vis_combo/{i:04d}.jpg")

  0%|          | 0/120 [00:00<?, ?it/s]

In [9]:
!ffmpeg -framerate 25 -pattern_type glob -i "vis_combo/*.jpg" -c:v libx264 -pix_fmt yuv420p vis_combo.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab